# Wikipedia Notable Life Expectancies 

## [Notebook 1 of 4:   Data Collection](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_collect_thanak_2022_06_10.ipynb)

## Context

This case study was chosen by this author as an end-to-end portfolio project for the following reasons:  
1. accessibility of the dataset
2. potential for use and demonstration of a wide range of skills, including coding in Python (using Pycharm and Jupyter Notebooks), version control (using Git, GitHub, and ReviewNB), web scraping (using Scrapy), relational database management (using SQLite and [SQLite Viewer](https://inloop.github.io/sqlite-viewer/)),  data cleaning, natural language processing (NLP using regular expressions and fuzzywuzzy), Exploratory Data Analysis (EDA using numpy, pandas, matplotlib, plotly, and seaborn), data preprocessing, unsupervised learning (Kmeans and hierachical clustering), supervised learning (regression models), user interface for predictions.
3. in particular, a dataset with potential for regression modeling was chosen to gain more exposure to its associated algorithms as this author had relative broader experience with classification models in past projects.
    

## Objective

The...

## Data Dictionary
Variable: Description

## Data Collection
- Data was collected from 6/9/22 to 6/10/22, using Scrapy. 

### 6/9/2022

- The [Wikipedia](https://en.wikipedia.org/wiki/Main_Page) page, [List of Deaths by Year](https://en.wikipedia.org/wiki/Lists_of_deaths_by_year), contains entries for as early as 1987, to the present day.  
- 1994 was chosen as the start year for collection as it is the first year with entries following the current format: "Name, age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, cause of death (if known), and reference."
- For ease of pagination, [Deaths in January 1994](https://en.wikipedia.org/wiki/Deaths_in_January_1994) was the start url for scraping, proceeding month by month through subsequent pages.
- Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py) scraped `month_year`, `day`, `name`, `info` ("age, country of citizenship at birth, subsequent country of citizenship (if applicable), reason for notability, and cause of death (if known)"), and `link` for each entry on each month's page.  
- The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table wp_deaths_94_to_22 within [wp_deaths_94_to_22.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_94_to_22.db).  This scraping was successful for January, 1994 through May, 2022 data.
- [Deaths in 2022 -- June](https://en.wikipedia.org/wiki/Deaths_in_2022#June)--the current month's page, at the time of scraping--varied in format, and was therefore scraped separately, after `num_references` was scraped for the previous entries.
- The original order of entries was preserved by Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py) in [wp_deaths_94_to_22.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_94_to_22.db).  Scrapy trades pagination order for speed, which is noticable when pagination is of higher magnitude.  Therefore, scraping each entry's page for number of references was done separately, as the order was sure to vary. Spider ["references"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/references.py) scraped for number of references.  The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table wp_reference_counts in [wp_reference_counts.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_reference_counts_2.db), with the results falling ~13,000 rows short of the 133,769 rows captured by Spider ["by_year"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/by_year.py).
- Finally, the [June, 2022](https://en.wikipedia.org/wiki/Deaths_in_2022#June) page was scraped by Spider ["June_2022"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/June_2022.py), successfully capturing all of the previous fields, including number of references.  Note that the number of pages was some order of magnitude smaller.
- The project's [pipelines.py](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/pipelines.py)* wrote results to SQLite table wp_deaths_June_2022 in [wp_deaths_June_2022.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_deaths_June_2022.db), resulting in 145 rows from the first part of June, 2022.

### 6/10/2022
The remaining data collection steps are outlined in this notebook:
1. [Reading, Sampling, and Checking Data Shape](#step1)
    - SQLite tables wp_deaths_94_to_22, wp_reference_counts, and wp_June_2022 were read in as pandas dataframes.
2. [Combining Dataframes](#step2)
    - Dataframes for wp_deaths_94_to_22 and wp_reference_counts were combined using `link` as the unique identifier.
    - Dataframe for wp_deaths_June_2022 was added.
3. [Duplicate Rows](#step3)
    - 9 rows of duplicate entries were dropped.
4. [Missing Values](#step4)
    - 5 rows missing essential data were dropped.
    - 6 entries had missing `name` and `num_references`, but contained the name in the `info` feature, to be extracted later during data cleaning.  As these entries had no associated page, their `num_references` was set equal to to 0.
5. [Missing `num_references` Values](#step5)
    - A single modification was made to the original XPath for the original "by_year" Spider, to match a variation on the pages for links with missing `num_references`.  
    - Those pages were then were rescraped iteratively by Spiders ["refs2"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs2.py), ["refs3"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs3.py), ["refs4"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs4.py), and ["refs5"](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/spiders/refs5.py).
    - The project's [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py)* wrote their respective results to SQLite tables refs2 in [refs2.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs2.db), refs3 in [refs3.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs3.db), refs4 in [refs4.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs4.db), and refs5 in [refs5.db](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/refs5.db).
    - During each iteration, the additional `num_references` were added to the main dataset and remaining links with missing `num_references` values were identified.
    - After the final iteration, 92 entries remained with missing `num_references` that were all missing an associated page, so `num_references` was set equal to to 0.  These entries contained all of the other relevant features, so were preserved.
    - The resultant raw dataset has 133,900 rows and 6 columns.

\*  The current version of [pipelines.py](https://github.com/teresahanak/wikipedia-notable-deaths/blob/main/wikipedia_notable_deaths/pipelines.py) reflects its use by the most recent project spider crawled, as it is reused for all spiders within the [Scrapy project folder](https://github.com/teresahanak/wikipedia-notable-deaths/tree/main/wikipedia_notable_deaths).

## Importing Necessary Libraries

In [1]:
# To structure code automatically
%load_ext nb_black

# To import/export sqlite databases
import sqlite3 as sql

# To help with reading and manipulating data
import pandas as pd

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
# To define the maximum number of rows to be displayed in a dataframe
pd.set_option("display.max_rows", 200)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To supress warnings
# import warnings

# warnings.filterwarnings("ignore")

# To set some visualization attributes
pd.set_option("max_colwidth", 150)

<IPython.core.display.Javascript object>

## Data Overview

<a id='step1'></a>
### Reading, Sampling, and Checking Data Shape

### January 1994 through May 2022 Data (without reference counts)

In [2]:
# Reading the wp_deaths_94_to_22 dataset from sql db and table
conn = sql.connect("wp_deaths_94_to_22.db")
raw_94_to_22 = pd.read_sql("SELECT * FROM wp_deaths_94_to_22", conn)

# Making a working copy
df_94_to_22 = raw_94_to_22.copy()

# Checking the shape
print(f"There are {df_94_to_22.shape[0]} rows and {df_94_to_22.shape[1]} columns.")

# Checking first 2 rows of the data
df_94_to_22.head(2)

There are 133769 rows and 5 columns.


,month_year,day,name,info,link
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer)
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty


<IPython.core.display.Javascript object>

In [3]:
# Checking last 2 rows of the data
df_94_to_22.tail(2)

,month_year,day,name,info,link
133767,May 2022,31,Dave Smith,", 72, American sound engineer, founder of Sequential.",https://en.wikipedia.org/wiki/Dave_Smith_(engineer)
133768,May 2022,31,Wang Zherong,", 86, Chinese tank designer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Wang_Zherong


<IPython.core.display.Javascript object>

In [4]:
# Checking a sample of the data
df_94_to_22.sample(5)

,month_year,day,name,info,link
82953,May 2016,13,Jan Korger,", 78, Czech physician and politician, member of the House of Peoples of the Federal Assembly of Czechoslovakia (1992).",https://en.wikipedia.org/wiki/Jan_Korger
40226,February 2009,28,Tom Sturdivant,", 78, American baseball player (New York Yankees).",https://en.wikipedia.org/wiki/Tom_Sturdivant
52398,November 2011,5,George Ansbro,", 96, American radio announcer.",https://en.wikipedia.org/wiki/George_Ansbro
69358,May 2014,22,Ruth Ziolkowski,", 87, American museum executive, CEO of the Crazy Horse Memorial (since 1982), cancer.",https://en.wikipedia.org/wiki/Ruth_Ziolkowski
12618,November 1998,22,Dariush Forouhar,", 69-70, Iranian pan-Iranist politician, murdered.",https://en.wikipedia.org/wiki/Dariush_Forouhar


<IPython.core.display.Javascript object>

#### Observations:
- There are 133,769 rows and 5 columns in the data from January, 1994 through May, 2022.
- The number of references was scraped separately.

### January 1994 through May 2022 Reference Count Data

In [5]:
# Reading the wp_reference_counts_2 dataset from sql db and table
conn = sql.connect("wp_reference_counts_2.db")
raw_reference_counts = pd.read_sql("SELECT * FROM wp_reference_counts_2", conn)

# Making a working copy
df_reference_counts = raw_reference_counts.copy()

# Checking the shape
print(
    f"There are {df_reference_counts.shape[0]} rows and {df_reference_counts.shape[1]} columns."
)

# Checking first 2 rows of the data
df_reference_counts.head(2)

There are 120368 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Lys_Gauty,5
1,https://en.wikipedia.org/wiki/William_Chappell_(dancer),21


<IPython.core.display.Javascript object>

In [6]:
# Checking last 2 rows of the data
df_reference_counts.tail(2)

,link,num_references
120366,https://en.wikipedia.org/wiki/Shirley_Thomas_(USC_professor),6
120367,https://en.wikipedia.org/wiki/James_Doohan,52


<IPython.core.display.Javascript object>

In [7]:
# Checking a sample of the data
df_reference_counts.sample(5)

,link,num_references
101630,https://en.wikipedia.org/wiki/Watkins_Abbitt,8
105851,https://en.wikipedia.org/wiki/Rex_Tucker,6
111796,https://en.wikipedia.org/wiki/Ben_Webb_(journalist),1
4720,https://en.wikipedia.org/wiki/Len_Lawson,7
114369,https://en.wikipedia.org/wiki/Gene_Burns,17


<IPython.core.display.Javascript object>

#### Observations:
- Here, we see that there are ~13,000 fewer rows for the reference data, indicating some pages were not successfully scraped to obtain the number of references for the individual.
- After combining the three dataframes, we can examine those pages and reattempt scraping them, in order to obtain the missing information.

### June 2022 Data

In [8]:
# Reading the wp_deaths_June_2022 dataset from sql db and table
conn = sql.connect("wp_deaths_June_2022.db")
raw_June_2022 = pd.read_sql("SELECT * FROM wp_deaths_June_2022", conn)

# Making a working copy
df_June_2022 = raw_June_2022.copy()

# Checking the shape
print(f"There are {df_June_2022.shape[0]} rows and {df_June_2022.shape[1]} columns.")

# Checking first 2 rows of the data
df_June_2022.head(2)

There are 145 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,June 2022,8,Mladen Frančić,", 67, Croatian football player and manager (Vrbovec, Podravina, Al-Watani Club).",https://en.wikipedia.org/wiki/Mladen_Fran%C4%8Di%C4%87,1
1,June 2022,6,Valery Ryumin,", 82, Russian cosmonaut (Soyuz 25, Soyuz 32, Soyuz 35).",https://en.wikipedia.org/wiki/Valery_Ryumin,2


<IPython.core.display.Javascript object>

#### Observations:
- The June, 2022 data does not follow the same row order as the previous dataframe, which was in order of day of the month. 
- For continuity, before concatinating the two dataframes, we will sort June, 2022 by day.

In [9]:
# Sorting by day
df_June_2022.sort_values(by="day", inplace=True)

# Re-checking first 2 rows of the data
df_June_2022.head(2)

,month_year,day,name,info,link,num_references
26,June 2022,1,Richard Oldcorn,", 84, English Olympic fencer (1964, 1968, 1972).",https://en.wikipedia.org/wiki/Richard_Oldcorn,5
20,June 2022,1,István Szőke,", 75, Hungarian footballer (Ferencváros, national team), stroke.",https://en.wikipedia.org/wiki/Istv%C3%A1n_Sz%C5%91ke,2


<IPython.core.display.Javascript object>

In [10]:
# Checking last 2 rows of the data
df_June_2022.tail(2)

,month_year,day,name,info,link,num_references
8,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
5,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [11]:
# Checking a sample of the data
df_June_2022.sample(5)

,month_year,day,name,info,link,num_references
64,June 2022,3,Geoff Hunter,", 62, English footballer (Crewe Alexandra, Port Vale, Wrexham).",https://en.wikipedia.org/wiki/Geoff_Hunter_(footballer),10
30,June 2022,1,Aleksandr Berketov,", 46, Russian footballer (Rotor Volgograd, CSKA Moscow).",https://en.wikipedia.org/wiki/Aleksandr_Berketov,2
122,June 2022,7,Jean-Louis Schefer,", 83, French art historian.",https://en.wikipedia.org/wiki/Jean-Louis_Schefer,1
53,June 2022,3,John Porter,", 87, American politician, member of the Illinois House of Representatives (1973–1979) and U.S. House of Representatives (1980–2001).",https://en.wikipedia.org/wiki/John_Porter_(Illinois_politician),5
28,June 2022,1,Jean Lèques,", 90, French politician, president of the Government of New Caledonia (1999–2001) and mayor of Nouméa (1986–2014).",https://en.wikipedia.org/wiki/Jean_L%C3%A8ques,4


<IPython.core.display.Javascript object>

#### Observations:
- Now, we are ready to combine the three dataframes.

<a id='step2'></a>
## Combining Dataframes

### Adding Number of References to 1994 through May 2022 Data

In [12]:
# Adding num_references column to 1994 through May 2022 data
df_combined = pd.merge(df_94_to_22, df_reference_counts, how="left", on="link")

# Checking first 2 rows of the data
df_combined.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

### Adding June 2022 Data

In [13]:
# Adding Juned 2022 data
df_combined = pd.concat([df_combined, df_June_2022], ignore_index=True)

# Making a working copy
df = df_combined.copy()

# Checking the shape
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns.")

# Checking first 2 rows of the data
df.head(2)

There are 133914 rows and 6 columns.


,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

In [14]:
# Checking last 2 rows of the data
df.tail(2)

,month_year,day,name,info,link,num_references
133912,June 2022,9,Oleg Moliboga,", 69, Russian volleyball player, Olympic champion (1980) and coach.",https://en.wikipedia.org/wiki/Oleg_Moliboga,2
133913,June 2022,9,Zou Jing,", 86, Chinese engineer, member of the Chinese Academy of Engineering.",https://en.wikipedia.org/wiki/Zou_Jing_(engineer),3


<IPython.core.display.Javascript object>

In [15]:
# Checking a sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references
46345,July 2010,12,Olga Guillot,", 87, Cuban singer, infarction.",https://en.wikipedia.org/wiki/Olga_Guillot,7
53446,January 2012,13,Guido Dessauer,", 96, German paper engineer and art collector.",https://en.wikipedia.org/wiki/Guido_Dessauer,14
63528,July 2013,18,Vaali,", 81, Indian Tamil lyricist, writer, poet and actor.",https://en.wikipedia.org/wiki/Vaali_(poet),9
104197,May 2019,14,Leopoldo Brizuela,", 55, Argentine journalist, translator and poet, winner of Alfaguara Prize (2012) and Konex Award (2014).",https://en.wikipedia.org/wiki/Leopoldo_Brizuela,6
68053,March 2014,16,Denké Kossi Wazo,", 55, Togolese footballer.",https://en.wikipedia.org/wiki/Denk%C3%A9_Kossi_Wazo,NaN


<IPython.core.display.Javascript object>

#### Confirming Correct Number of Resultant Entries

In [16]:
# Confirming correct number of total rows
df_94_to_22.shape[0] + df_June_2022.shape[0]

del df_94_to_22, df_reference_counts, df_June_2022

<IPython.core.display.Javascript object>

#### Observations:
- We have successfully combined the three dataframes.
- Now, we can check for data types, duplicates, and missing values.

<a id='step3'></a>
## Checking Data Types, Duplicates, and Null Values

### Data types

In [17]:
# Checking data types and null values
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133914 entries, 0 to 133913
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133914 non-null  object
 1   day             133914 non-null  object
 2   name            133903 non-null  object
 3   info            133914 non-null  object
 4   link            133914 non-null  object
 5   num_references  120637 non-null  object
dtypes: object(6)
memory usage: 6.1+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are 6 columns, all of object type.
- `name` and `num_references` both have missing values.
- The data is in a very raw format and there are other columns that have combined information that will need to be extracted.
- For now, we will leave all as object type.

<a id='step3'></a>
### Duplicate Rows

In [18]:
# Checking duplicate rows
df.duplicated().sum()

9

<IPython.core.display.Javascript object>

#### Observations:
- There are 9 duplicate rows that we will drop now.

In [19]:
# Drop duplicate rows
df.drop_duplicates(inplace=True, ignore_index=True)

# Re-check shape
df.shape

(133905, 6)

<IPython.core.display.Javascript object>

<a id='step4'></a>
### Missing Values

In [20]:
# Check percentage of null values by column
df.isnull().sum() / df.count() * 100

month_year        0.000
day               0.000
name              0.008
info              0.000
link              0.000
num_references   11.007
dtype: float64

<IPython.core.display.Javascript object>

In [21]:
# Checking number of missing values per row
df.isnull().sum(axis=1).value_counts()

0    120628
1     13266
2        11
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- The number of rows missing only 1 value appears consistent with our anticipated missing `num_references`.
- There are 11 rows that are each missing 2 values.  Let us take a closer look at these rows.

In [22]:
# Checking the rows that are missing values for 2 columns
missing_2 = df[df.isnull().sum(axis=1) == 2]
missing_2

,month_year,day,name,info,link,num_references
18937,August 2001,11,None,"Kevin Kowalcyk, 2, known for eating a hamburger contaminated with E. coli O157:H7.",https://en.wikipedia.orgNone,NaN
24985,January 2004,22,None,"Vincent Palmer, 37, British criminal.",https://en.wikipedia.orgNone,NaN
27458,March 2005,1,None,"Barry Stigler, 57, American voice actor.",https://en.wikipedia.orgNone,NaN
34077,July 2007,11,None,"Nana Gualdi, 75, German singer and actress.",https://en.wikipedia.orgNone,NaN
35097,November 2007,11,None,,https://en.wikipedia.orgNone,NaN
41075,May 2009,18,None,Either killed in a missile attack or shot:\n,https://en.wikipedia.orgNone,NaN
64771,September 2013,29,None,"Scott Workman, 47, American stuntman (, , ), cancer.",https://en.wikipedia.orgNone,NaN
76024,April 2015,29,None,Notable convicted drug traffickers executed by Indonesian firing squad:\n,https://en.wikipedia.orgNone,NaN
105871,August 2019,2,None,"Japanese convicted murderers, executed by hanging.\n",https://en.wikipedia.orgNone,NaN
106617,September 2019,12,None,"Thami Shobede, 31, Singer Songwriter",https://en.wikipedia.orgNone,NaN


<IPython.core.display.Javascript object>

#### Observations:
- We can see that multiple rows are missing `name`, but have the name in `info`, so we can extract it later.  
- The missing link itself is not of concern as it serves only as a means by which to retrieve the `num_references` value.
- As there is no associated link for the individual, we can safely replace the NaN `num_references` values for rows with extractable names with 0.
- We can proceed with removing the rows that lack an extractable name, as they also lack other information necessary for the analysis.

In [23]:
# List of rows to keep
keep_rows = [18937, 24985, 27458, 34077, 64771, 106617]

# For loop to replace num_references NaNs with 0 for rows with extractable names
for row in keep_rows:
    df.loc[row, "num_references"] = 0

# List of rows to remove
remove_rows = [index for index in missing_2.index if index not in keep_rows]
del missing_2

# Dropping rows
df.drop(remove_rows, inplace=True)

# Re-checking shape
df.shape

(133900, 6)

<IPython.core.display.Javascript object>

In [24]:
# Re-check info
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 133900 entries, 0 to 133904
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   month_year      133900 non-null  object
 1   day             133900 non-null  object
 2   name            133894 non-null  object
 3   info            133900 non-null  object
 4   link            133900 non-null  object
 5   num_references  120634 non-null  object
dtypes: object(6)
memory usage: 7.2+ MB


<IPython.core.display.Javascript object>

#### Observations:
- There are now only 6 rows with missing `name`, corresponding to the names we identified in `info`, that we will extract later.
- The remaining missing values are all for `num_references`, so we can proceed to make another attempt at scraping this information.
- Let us check a sample of these rows.

In [25]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
87258,January 2017,9,Terry Ramshaw,", 74, English rugby league footballer (Featherstone Rovers, Wakefield Trinity).",https://en.wikipedia.org/wiki/Terry_Ramshaw,NaN
94241,January 2018,8,Myron Rush,", 96, American Kremlinologist, Cornell University professor, and CIA analyst, kidney failure.",https://en.wikipedia.org/wiki/Myron_Rush,NaN
36278,February 2008,26,Dick Fletcher,", 65, American meteorologist at WTSP Tampa Bay, stroke.",https://en.wikipedia.org/wiki/Dick_Fletcher,NaN
35322,December 2007,5,Robert A. Hawkins,", 19, American mass murderer, suicide by gunshot.",https://en.wikipedia.org/wiki/Robert_A._Hawkins,NaN
65426,November 2013,7,Joey Manley,", 48, American website publisher (), pneumonia.",https://en.wikipedia.org/wiki/Joey_Manley,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links reveals that the pages contain references.  
- Therefore, they either have a variation in the XPath followed for scraping, or Scrapy had an issue with following their links.
- We will export a dataframe of the links to the pages that need to be re-scraped for `num_references`.

In [26]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df = df[df["num_references"].isna()]["link"]
rescrape_df.to_csv("rescrape_df.csv", index=False)

del rescrape_df

<IPython.core.display.Javascript object>

<a id='step5'></a>
## Missing `num_references` Values

### First Re-scrape with Spider ["refs2"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs2.py)

#### Observations:
- A second iteration of scraping individual pages for number of references reveals variation in the XPath for those pages.
- The new spider, ["refs2"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs2.py), used an updated XPath, with the identified variation.
- As before, we will import and merge the data, examine rows still missing `num_references` and re-scrape them, adjusting the spider's XPath again, if needed.

In [27]:
# Reading the refs2 dataset from sql db and table
conn = sql.connect("refs2.db")
raw_refs2 = pd.read_sql("SELECT * FROM refs2", conn)

# Making a working copy
df_refs2 = raw_refs2.copy()

# Checking the shape
print(f"There are {df_refs2.shape[0]} rows and {df_refs2.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs2.head(2)

There are 7365 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_American_supercentenarians#Charlotte_Benkner,63
1,https://en.wikipedia.org/wiki/Eugene_Record,11


<IPython.core.display.Javascript object>

In [28]:
# Checking last 2 rows of the data
df_refs2.tail(2)

,link,num_references
7363,https://en.wikipedia.org/wiki/Gunnar_Utterberg,3
7364,https://en.wikipedia.org/wiki/Bill_Sudakis,19


<IPython.core.display.Javascript object>

In [29]:
# Checking a sample of the data
df_refs2.sample(5)

,link,num_references
2140,https://en.wikipedia.org/wiki/Amalie_Christie,5
6659,https://en.wikipedia.org/wiki/Millie_Small,27
2584,https://en.wikipedia.org/wiki/Lillian_Adams,2
6232,https://en.wikipedia.org/wiki/Lewis_Ryder,1
2188,https://en.wikipedia.org/wiki/Ibrahima_Fofana,5


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 7365 of the missing values.

#### Adding Missing References to Dataframe

In [30]:
# Adding new num_references column to data
df = pd.merge(df, df_refs2, how="left", on="link")
del df_refs2

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
28614,August 2005,6,James Wilson,", 82, Irish composer.",https://en.wikipedia.org/wiki/James_Wilson_(composer),0,NaN
91,January 1994,14,Nubar Terziyan,", 84, Turkish actor.",https://en.wikipedia.org/wiki/Nubar_Terziyan,3,NaN
95230,February 2018,21,Ian Aitken,", 90, British journalist and political commentator.",https://en.wikipedia.org/wiki/Ian_Aitken_(journalist),NaN,7
66286,December 2013,25,John Rutherford,", 78, English cricketer.",https://en.wikipedia.org/wiki/John_Rutherford_(Cambridge_University_cricketer),4,NaN
26551,October 2004,28,Rosalind Hicks,", 85, British literary guardian and the only child of Agatha Christie.",https://en.wikipedia.org/wiki/Rosalind_Hicks,13,NaN


<IPython.core.display.Javascript object>

In [31]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
61173,March 2013,14,Edward Bland,", 86, American filmmaker and composer ().",https://en.wikipedia.org/wiki/Edward_Bland,1,NaN
47571,October 2010,21,José Carbajal,", 66, Uruguayan singer, guitarist, and composer (), cardiac arrest.",https://en.wikipedia.org/wiki/Jos%C3%A9_Carbajal_(Uruguayan_musician),2,NaN
18529,June 2001,17,Donald J. Cram,", 82, American chemist and co-winner of Nobel Prize in Chemistry in 1987.",https://en.wikipedia.org/wiki/Donald_J._Cram,10,NaN
90883,July 2017,11,George W. BonDurant,", 101, American preacher and academic administrator.",https://en.wikipedia.org/wiki/George_W._BonDurant,6,NaN
51439,August 2011,21,Budd Hopkins,", 80, American artist and UFO researcher, liver cancer.",https://en.wikipedia.org/wiki/Budd_Hopkins,74,NaN


<IPython.core.display.Javascript object>

In [32]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [33]:
# Checking remaining missing values
df.isna().sum()

month_year           0
day                  0
name                 6
info                 0
link                 0
num_references    5895
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have nearly 6000 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [34]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
56732,July 2012,21,Richard Jessup,", 87, American Olympic sailor.",https://en.wikipedia.org/wiki/Richard_Jessup_(sailor),NaN
42451,September 2009,22,Andrea Maltarolli,", 46, Brazilian telenovela screenwriter (), cancer.",https://en.wikipedia.org/wiki/Andrea_Maltarolli,NaN
68153,March 2014,23,Walter Ewbank,", 96, British Anglican prelate, Archdeacon of Westmorland and Furness (1971–1977), Archdeacon of Carlisle (1978–1984).",https://en.wikipedia.org/wiki/Walter_Ewbank,NaN
92398,October 2017,4,John Miller,", 79, American politician, member of the U.S. House of Representatives for Washington's 1st congressional district (1985–1993), cancer.",https://en.wikipedia.org/wiki/John_Miller_(Washington_politician),NaN
64195,August 2013,24,Sonia Coutinho,", 74, Brazilian journalist, short story writer and novelist, heart attack.",https://en.wikipedia.org/wiki/Sonia_Coutinho,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [35]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_2nd = df[df["num_references"].isna()]["link"]
rescrape_df_2nd.to_csv("rescrape_df_2nd.csv", index=False)
del rescrape_df_2nd

<IPython.core.display.Javascript object>

### Second Re-scrape with Spider ["refs3"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs3.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [36]:
# Reading the refs3 dataset from sql db and table
conn = sql.connect("refs3.db")
raw_refs3 = pd.read_sql("SELECT * FROM refs3", conn)

# Making a working copy
df_refs3 = raw_refs3.copy()

# Checking the shape
print(f"There are {df_refs3.shape[0]} rows and {df_refs3.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs3.head(2)

There are 3633 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_American_supercentenarians#Grace_Thaxton,63
1,"https://en.wikipedia.org/wiki/Christopher_Prout,_Baron_Kingsland",6


<IPython.core.display.Javascript object>

In [37]:
# Checking last 2 rows of the data
df_refs3.tail(2)

,link,num_references
3631,https://en.wikipedia.org/wiki/Concepci%C3%B3n_Ram%C3%ADrez,8
3632,https://en.wikipedia.org/wiki/Mary_Mahoney_(physician),6


<IPython.core.display.Javascript object>

In [38]:
# Checking a sample of the data
df_refs3.sample(5)

,link,num_references
1428,https://en.wikipedia.org/wiki/Paul_O%27Connor_(hurler),2
2848,https://en.wikipedia.org/wiki/Fran_Bera,5
603,https://en.wikipedia.org/wiki/Jorge_Mayer,0
463,https://en.wikipedia.org/wiki/Bobby_Thomson,17
3197,https://en.wikipedia.org/wiki/Andr%C3%A9e_Lachapelle,5


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 3633 of the missing values.

#### Adding Missing References to Dataframe

In [39]:
# Adding new num_references column to data
df = pd.merge(df, df_refs3, how="left", on="link")
del df_refs3

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
26340,September 2004,22,Ray Traylor,", 42, American professional wrestler known as .",https://en.wikipedia.org/wiki/Big_Boss_Man,44,NaN
118081,December 2020,5,Anusuya Prasad Maikhuri,", 59, Indian politician, Uttarakhand MLA (2002–2007, 2012–2017), complications from COVID-19.",https://en.wikipedia.org/wiki/Anusuya_Prasad_Maikhuri,NaN,NaN
124355,June 2021,14,Sir Eion Edgar,", 76, New Zealand businessman and philanthropist, chancellor of the University of Otago (1999–2003), pancreatic cancer.",https://en.wikipedia.org/wiki/Eion_Edgar,10,NaN
95814,March 2018,19,Keith O'Brien,", 80, Scottish Roman Catholic Cardinal, Archbishop of St Andrews and Edinburgh (1985–2013), complications from a fall.",https://en.wikipedia.org/wiki/Keith_O%27Brien,101,NaN
44319,February 2010,13,Werner Forman,", 89, Czech-born British photographer.",https://en.wikipedia.org/wiki/Werner_Forman,0,NaN


<IPython.core.display.Javascript object>

In [40]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
108668,December 2019,24,Kelly Fraser,", 26, Canadian Inuktitut pop singer and songwriter, suicide.",https://en.wikipedia.org/wiki/Kelly_Fraser,14,NaN
114646,July 2020,21,Jos Bax,", 74, Dutch footballer (FC Eindhoven, Helmond Sport, VVV-Venlo).",https://en.wikipedia.org/wiki/Jos_Bax,3,NaN
132014,March 2022,14,Terry Dunleavy,", 93, New Zealand winemaker.",https://en.wikipedia.org/wiki/Terry_Dunleavy,3,NaN
8515,April 1997,18,Bob Carpenter,", 79, American basketball player.",https://en.wikipedia.org/wiki/Bob_Carpenter_(basketball),0,NaN
203,February 1994,2,Yōichi Nakanishi,", 76, Japanese politician and Governor of Ishikawa Prefecture.",https://en.wikipedia.org/wiki/Y%C5%8Dichi_Nakanishi,0,NaN


<IPython.core.display.Javascript object>

In [41]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [42]:
# Checking remaining missing values
df.isna().sum()

month_year           0
day                  0
name                 6
info                 0
link                 0
num_references    2260
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have 2260 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [43]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
103600,April 2019,16,Bill LeCaine,", 81, Canadian ice hockey player (Pittsburgh Penguins).",https://en.wikipedia.org/wiki/Bill_LeCaine,NaN
112957,May 2020,17,Mercedes Mendoza Suasti,", 93, Ecuadorian pasillo singer.",https://en.wikipedia.org/wiki/Mercedes_Mendoza_Suasti,NaN
103174,March 2019,26,Michel Bacos,", 95, French pilot (Air France Flight 139).",https://en.wikipedia.org/wiki/Michel_Bacos,NaN
92702,October 2017,22,Walter Babington Thomas,", 98, New Zealand-born British army officer, GOC Far East Land Forces (1970–1971).",https://en.wikipedia.org/wiki/Walter_Babington_Thomas,NaN
95310,February 2018,25,Leif Liljeroth,", 93, Swedish actor.",https://en.wikipedia.org/wiki/Leif_Liljeroth,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [44]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_3rd = df[df["num_references"].isna()]["link"]
rescrape_df_3rd.to_csv("rescrape_df_3rd.csv", index=False)
del rescrape_df_3rd

<IPython.core.display.Javascript object>

### Third Re-scrape with Spider ["refs4"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs4.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [45]:
# Reading the refs4 dataset from sql db and table
conn = sql.connect("refs4.db")
raw_refs4 = pd.read_sql("SELECT * FROM refs4", conn)

# Making a working copy
df_refs4 = raw_refs4.copy()

# Checking the shape
print(f"There are {df_refs4.shape[0]} rows and {df_refs4.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs4.head(2)

There are 1960 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/List_of_French_supercentenarians,157
1,https://en.wikipedia.org/wiki/Bob_Kahler,3


<IPython.core.display.Javascript object>

In [46]:
# Checking last 2 rows of the data
df_refs4.tail(2)

,link,num_references
1958,https://en.wikipedia.org/wiki/Jorge_Sampaio,73
1959,https://en.wikipedia.org/wiki/Irma_Kalish,14


<IPython.core.display.Javascript object>

In [47]:
# Checking a sample of the data
df_refs4.sample(5)

,link,num_references
853,https://en.wikipedia.org/wiki/James_Carroll_(actor),3
1065,https://en.wikipedia.org/wiki/Jean_Worthley,2
558,https://en.wikipedia.org/wiki/Rositsa_Yanakieva,1
997,https://en.wikipedia.org/wiki/Bruce_Mazlish,31
1301,https://en.wikipedia.org/wiki/Warren_Hawksley,5


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 1960 of the missing values.

#### Adding Missing References to Dataframe

In [48]:
# Adding new num_references column to data
df = pd.merge(df, df_refs4, how="left", on="link")
del df_refs4

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
86248,November 2016,17,Don Waller,", 65, American music journalist and singer, lung cancer.",https://en.wikipedia.org/wiki/Don_Waller,3,NaN
130804,February 2022,1,Bud Clark,", 90, American politician, mayor of Portland, Oregon (1985–1992), heart failure.",https://en.wikipedia.org/wiki/Bud_Clark,27,NaN
16974,November 2000,17,Bim Sherman,", 50, Jamaican musician.",https://en.wikipedia.org/wiki/Bim_Sherman,3,NaN
117245,November 2020,5,Govardhan Upadhyay,", 77, Indian politician, Madhya Pradesh MLA (1985–1990, 2013–2018).",https://en.wikipedia.org/wiki/Govardhan_Upadhyay,4,NaN
15966,May 2000,30,Iko Carreira,", 66, Angolan army general and politician.",https://en.wikipedia.org/wiki/Iko_Carreira,3,NaN


<IPython.core.display.Javascript object>

In [49]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
17401,January 2001,15,Alex Blignaut,", 68, South African racing driver and racing team owner, domestic accident.",https://en.wikipedia.org/wiki/Alex_Blignaut,3,NaN
82561,April 2016,22,Robert Price,", 83, American attorney and political campaign manager.",https://en.wikipedia.org/wiki/Robert_Price_(attorney),8,NaN
111421,April 2020,3,Alexander A. Gurshtein,", 83, Russian astronomer.",https://en.wikipedia.org/wiki/Alexander_A._Gurshtein,3,NaN
123468,May 2021,12,Ton Pansier,", 74, Dutch footballer (XerxesDZB, SV SVV).",https://en.wikipedia.org/wiki/Ton_Pansier,8,NaN
80943,February 2016,3,Ernesto Alais,", 86, Argentine Olympic sports shooter.",https://en.wikipedia.org/wiki/Ernesto_Alais,1,NaN


<IPython.core.display.Javascript object>

In [50]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [51]:
# Checking remaining missing values
df.isna().sum()

month_year          0
day                 0
name                6
info                0
link                0
num_references    300
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have just 300 remaining missing values for `num_references`, so we will iterate through the rescraping again.

In [52]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()].sample(5)

,month_year,day,name,info,link,num_references
117602,November 2020,17,John Poole,", 87, English footballer (Port Vale, Macclesfield Town).","https://en.wikipedia.org/wiki/John_Poole_(footballer,_born_1932)",NaN
120625,February 2021,15,Nechan Karakéhéyan,", 88, Greek-born Armenian Catholic hierarch, ordinary of Eastern Europe (2005–2010) and Greece (1991–2000), eparch of Ishafan (2001–2003).",https://en.wikipedia.org/wiki/Nechan_Karak%C3%A9h%C3%A9yan,NaN
133430,May 2022,16,Lenny Laroux,", Dutch musician and singer.",https://en.wikipedia.org/w/index.php?title=Lenny_Laroux&action=edit&redlink=1,NaN
120297,February 2021,6,Rajie Cook,", 90, American graphic designer.",https://en.wikipedia.org/wiki/Rajie_Cook,NaN
133631,May 2022,26,Lee Eol,", 58, South Korean actor (, , ).",https://en.wikipedia.org/w/index.php?title=Lee_Eol&action=edit&redlink=1,NaN


<IPython.core.display.Javascript object>

#### Observations:
- Following the links again reveals that the pages contain references.  
- We will export another dataframe of the links to the pages that need to be re-scraped for `num_references` and examine the pages for alternate XPaths for scraping.

In [53]:
# Exporting dataframe of pages to rescrape for num_references
rescrape_df_4th = df[df["num_references"].isna()]["link"]
rescrape_df_4th.to_csv("rescrape_df_4th.csv", index=False)
del rescrape_df_4th

<IPython.core.display.Javascript object>

### Fourth Re-scrape with Spider ["refs5"](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wikipedia_life_expectancy/spiders/refs5.py)

#### Observations:
- The XPath matched that of the last scraping iteration for several pages, so the scraping was repeated for the remaining rows with missing `num_references`.

In [54]:
# Reading the refs5 dataset from sql db and table
conn = sql.connect("refs5.db")
raw_refs5 = pd.read_sql("SELECT * FROM refs5", conn)

# Making a working copy
df_refs5 = raw_refs5.copy()

# Checking the shape
print(f"There are {df_refs5.shape[0]} rows and {df_refs5.shape[1]} columns.")

# Checking first 2 rows of the data
df_refs5.head(2)

There are 208 rows and 2 columns.


,link,num_references
0,https://en.wikipedia.org/wiki/Mohamed_Haytham_Khayat,5
1,https://en.wikipedia.org/wiki/Sid_McCray,6


<IPython.core.display.Javascript object>

In [55]:
# Checking last 2 rows of the data
df_refs5.tail(2)

,link,num_references
206,https://en.wikipedia.org/wiki/Anna_Cataldi,9
207,https://en.wikipedia.org/wiki/Allan_Egolf,4


<IPython.core.display.Javascript object>

In [56]:
# Checking a sample of the data
df_refs5.sample(5)

,link,num_references
107,https://en.wikipedia.org/wiki/Frank_Baude,3
80,https://en.wikipedia.org/wiki/Carlo_Nayaradou,2
98,https://en.wikipedia.org/wiki/Bjarne_Holen,2
51,"https://en.wikipedia.org/wiki/John_Poole_(footballer,_born_1932)",10
201,https://en.wikipedia.org/wiki/Albert_Kakou_Tiapani,1


<IPython.core.display.Javascript object>

#### Observations:
- We were able to obtain 208 of the missing values.

#### Adding Missing References to Dataframe

In [57]:
# Adding new num_references column to data
df = pd.merge(df, df_refs5, how="left", on="link")
del df_refs5

# Checking sample of the data
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
52267,October 2011,27,Tom Brown,", 89, American tennis player.",https://en.wikipedia.org/wiki/Tom_Brown_(tennis),6,NaN
120847,February 2021,22,Seif Bamporiki,", Rwandan politician, shot.",https://en.wikipedia.org/wiki/Seif_Bamporiki,3,NaN
68537,April 2014,11,Carl Zimmermann,", 96, American news anchor (WITI) and World War II war correspondent.",https://en.wikipedia.org/wiki/Carl_Zimmermann_(news_anchor),1,NaN
67345,February 2014,13,Richard Møller Nielsen,", 76, Danish football player and manager, brain tumour.",https://en.wikipedia.org/wiki/Richard_M%C3%B8ller_Nielsen,4,NaN
103159,March 2019,25,Miklós Martin,", 87, Hungarian Olympic water polo player.",https://en.wikipedia.org/wiki/Mikl%C3%B3s_Martin,11,NaN


<IPython.core.display.Javascript object>

In [58]:
# Filling missing values with newly obtained values
df["num_references_x"].fillna(df["num_references_y"], inplace=True)
df.sample(5)

,month_year,day,name,info,link,num_references_x,num_references_y
81143,February 2016,13,Angela Bairstow,", 73, English badminton player.",https://en.wikipedia.org/wiki/Angela_Bairstow,6,NaN
29446,November 2005,28,Eric Nance,", 45, American convicted murderer, executed in Arkansas.",https://en.wikipedia.org/wiki/Eric_Nance,6,NaN
88347,February 2017,27,John Harlan,", 91, American radio and television personality (, ).",https://en.wikipedia.org/wiki/John_Harlan_(announcer),5,NaN
34624,September 2007,16,Jean Balissat,", 71, Swiss musician.",https://en.wikipedia.org/wiki/Jean_Balissat,0,NaN
49593,March 2011,30,Liaquat Soldier,", 59, Pakistani comedian, heart attack.",https://en.wikipedia.org/wiki/Liaquat_Soldier,4,NaN


<IPython.core.display.Javascript object>

In [59]:
# Dropping new references column and reverting to original column name
df.drop("num_references_y", axis=1, inplace=True)
df.rename(columns={"num_references_x": "num_references"}, inplace=True)
df.head(2)

,month_year,day,name,info,link,num_references
0,January 1994,1,William Chappell,", 86, British dancer, ballet designer and director.",https://en.wikipedia.org/wiki/William_Chappell_(dancer),21
1,January 1994,1,Raymond Crotty,", 68, Irish economist, writer, and academic.",https://en.wikipedia.org/wiki/Raymond_Crotty,12


<IPython.core.display.Javascript object>

#### Checking Remaining Missing Values

In [60]:
# Checking remaining missing values
df.isna().sum()

month_year         0
day                0
name               6
info               0
link               0
num_references    92
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- We have just 92 remaining missing values for `num_references`.
- Let us examine the remaining rows with missing values more closely.

In [61]:
# Checking sample of rows missing num_references
df[df["num_references"].isna()]

,month_year,day,name,info,link,num_references
12476,October 1998,29,Sjoerdtsje Faber,", 83, Dutch speed skater.",https://en.wikipedia.org/w/index.php?title=Sjoerdje_Faber&action=edit&redlink=1,NaN
15859,May 2000,10,Bill Foster,", 68, American entertainer.",https://en.wikipedia.org/w/index.php?title=Bill_Foster_(performer)&action=edit&redlink=1,NaN
18561,June 2001,20,Demetreus Nix,"Douglas Scott, 20, High-school student murdered by .",https://en.wikipedia.org/w/index.php?title=Demetreus_Nix&action=edit&redlink=1,NaN
19905,December 2001,18,Sietske Pasveer,", 86, Dutch speed skater.",https://en.wikipedia.org/w/index.php?title=Sietske_Pasveer&action=edit&redlink=1,NaN
22942,March 2003,3,Gilbert Wheeler Beebe,", 90, American epidemiologist and statistician, conducted ground-breaking radiation exposure studies.",https://en.wikipedia.org/w/index.php?title=Gilbert_Wheeler_Beebe&action=edit&redlink=1,NaN
23291,April 2003,27,Charles A. Marvin,", 73, American district attorney and judge.",https://en.wikipedia.org/w/index.php?title=Charles_A._Marvin&action=edit&redlink=1,NaN
24346,October 2003,15,Ray Kuhlman,", 84, American pilot and businessman.",https://en.wikipedia.org/w/index.php?title=Ray_Kuhlman&action=edit&redlink=1,NaN
33653,May 2007,20,Moses Siregar,", 96, Maritime Chef and American World War 2 Veteran",https://en.wikipedia.org/w/index.php?title=Moses_Siregar&action=edit&redlink=1,NaN
35963,January 2008,31,Michael A. Dions,", 90, American Olympic swimmer, natural causes",https://en.wikipedia.org/w/index.php?title=Michael_A._Dions&action=edit&redlink=1,NaN
36198,February 2008,19,Samuel Champkin,", 28, American singer in Metal band Tech Giants, car crash.",https://en.wikipedia.org/w/index.php?title=Samuel_Champkin&action=edit&redlink=1,NaN


<IPython.core.display.Javascript object>

#### Observations:
- None of the remaining links direct to a personal page for the individual, as such a page does not yet exist.
- As such, as the rows contain the other elements necessary for analysis, it is safe to replace the missing `num_references` values with 0, for these rows.

In [62]:
# Fill remaining missing values for num_references with 0
df["num_references"].fillna(0, inplace=True)

# Recheck missing values
df.isna().sum()

month_year        0
day               0
name              6
info              0
link              0
num_references    0
dtype: int64

<IPython.core.display.Javascript object>

#### Observations:
- All of the missing values for `num_references` have been addressed.
- The 6 remaining missing values for `name` will be fixed during data cleaning.
- We now have our complete raw dataset, which we will also export to a SQLite database.
- Then it is time to start cleaning the data.

### Exporting Dataset to SQLite Database [wp_life_expect_raw_complete.db](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_raw_complete.db)

In [63]:
# Saving complete raw dataset in a SQLite database
conn = sql.connect("wp_life_expect_raw_complete.db")
df.to_sql("wp_life_expect_raw_complete", conn, index=False)

133900

<IPython.core.display.Javascript object>

# [Proceed to Notebook 2 of  4:  Data Cleaning](https://github.com/teresahanak/wikipedia-life-expectancy/blob/main/wp_life_expect_data_clean_thanak_2022_06_13.ipynb)